<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_PortfolioAllocation_NeurIPS_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math
from datetime import datetime
from Data_processor import YahooFinanceProcessor

### Download Data

In [2]:
# # 設定資料日期範圍
# HISTORICAL_START_DATE = "2012-06-01"
# HISTORICAL_END_DATE = "2018-12-31"

# TRAIN_START_DATE = "2012-06-01"
# TRAIN_END_DATE = "2018-06-30"

# FIX_START_DATE = "2018-01-01"
# FIX_END_DATE = "2019-06-30"

# VALIDATION_START_DATE = "2019-01-01"
# VALIDATION_END_DATE = "2019-12-31"

# TEST_START_DATE = "2020-01-01"
# TEST_END_DATE = "2020-06-30"

### Data Preprocessing

In [3]:
# # 資料下載與前處理(只包含training範圍的資料)

# # 股票代碼與技術指標的list
# # 資料下載與前處理(只包含training範圍的資料)

# # 股票代碼與技術指標的list
# ticker_list = ['AAPL','MSFT','AMZN','TSLA','GOOGL','NVDA','BRK-A','META','UNH','JNJ ']
# tech_list = []

# # # 技術指標與VIX的開關，如果不想加就設定為False
# if_vix = False
# if_tech = False

# days = [1, 5, 20, 60, 125, 250]

# # # 下載並清洗資料
# DP = YahooFinanceProcessor()
# historical_data = DP.download_data(start_date = HISTORICAL_START_DATE,end_date = HISTORICAL_END_DATE,ticker_list = ticker_list, time_interval='1D')
# historical_data = DP.clean_data(historical_data)
# train_data = DP.download_data(start_date = TRAIN_START_DATE,end_date = TRAIN_END_DATE,ticker_list = ticker_list, time_interval='1D')
# train_data = DP.clean_data(train_data)
# fix_data = DP.download_data(start_date = FIX_START_DATE,end_date = FIX_END_DATE,ticker_list = ticker_list, time_interval='1D')
# fix_data = DP.clean_data(fix_data)
# validation_data = DP.download_data(start_date = VALIDATION_START_DATE,end_date = VALIDATION_END_DATE,ticker_list = ticker_list, time_interval='1D')
# validation_data = DP.clean_data(validation_data)
# test_data = DP.download_data(start_date = TEST_START_DATE,end_date = TEST_END_DATE,ticker_list = ticker_list, time_interval='1D')
# test_data = DP.clean_data(test_data)

# # # DataFrame轉成np.array
# price_array_historical, tech_array_train, turbulence_array_train = DP.df_to_array(historical_data,tech_list, if_vix, if_tech)
# price_array_train, tech_array_train, turbulence_array_train = DP.df_to_array(train_data,tech_list, if_vix, if_tech)
# price_array_fix, tech_array_train, turbulence_array_train = DP.df_to_array(fix_data,tech_list, if_vix, if_tech)
# price_array_validation, tech_array_train, turbulence_array_train = DP.df_to_array(validation_data,tech_list, if_vix, if_tech)
# price_array_test, tech_array_test, turbulence_array_test = DP.df_to_array(test_data,tech_list, if_vix, if_tech)

# aug_state_array_train = np.zeros((0,0))
# aug_state_array_test = np.zeros((0,0))

# # 加入技術指標和恐慌指數(如果有的話)
# # if if_tech:
# #     tech_list = ['close_30_sma','close_60_sma','macd','boll_ub','boll_lb','dx_30','rsi_30']
# #     train_data = DP.add_technical_indicator(train_data, tech_list)
# #     test_data = DP.add_technical_indicator(test_data, tech_list)
    
# # if if_vix:
# #     train_data = DP.add_vix(train_data)
# #     test_data = DP.add_vix(test_data)
    
# #     print(all_env_price)
# #     all_env_tech.append(tech_array_train[start_day:start_day+200, :])
# #     print(all_env_tech)
# #     all_env_turbulence.append(turbulence_array_train[start_day:start_day+200])
# #     print(all_env_turbulence)
   
# # 加入額外的state資訊 ( RN embedding )

# # aug_state_array = np.zeros((price_array.shape[0],11,8)) # (754,11,8)
# # 印一下形狀
# #print('=============')
# #print('price_array_train: ',price_array_train.shape)

In [4]:
# #存csv
# pd.DataFrame(price_array_historical).to_csv('./data/historical.csv',index=False)
# pd.DataFrame(price_array_train).to_csv('./data/train.csv',index=False)
# pd.DataFrame(price_array_fix).to_csv('./data/fix.csv',index=False)
# pd.DataFrame(price_array_validation).to_csv('./data/validation.csv',index=False)
# pd.DataFrame(price_array_test).to_csv('./data/test.csv',index=False)

In [5]:
#讀csv
price_array_historical = pd.read_csv('./data/historical.csv').to_numpy()
price_array_train = pd.read_csv('./data/train.csv').to_numpy()
price_array_fix = pd.read_csv('./data/fix.csv').to_numpy()
price_array_validation = pd.read_csv('./data/validation.csv').to_numpy()
price_array_test = pd.read_csv('./data/test.csv').to_numpy()

In [6]:
#定義
ticker_list = ['AAPL','MSFT','AMZN','TSLA','GOOGL','NVDA','BRK-A','META','UNH','JNJ ']
days = [1, 5, 20, 60, 125, 250]
validation_day = price_array_validation.shape[0]
aug_state_array_train = np.zeros((0,0))
aug_state_array_test = np.zeros((0,0))
tech_array_test = []

In [7]:
#price_return = pd.DataFrame(price_array[self.number]).pct_change(1).fillna(0).values
#print('self.price_return',self.price_return.shape)
price_historical_return = pd.DataFrame(price_array_historical).pct_change(1).fillna(0).values
price_train_return = pd.DataFrame(price_array_train).pct_change(1).fillna(0).values
price_fix_return = pd.DataFrame(price_array_fix).pct_change(1).fillna(0).values
price_validation_return = pd.DataFrame(price_array_validation).pct_change(1).fillna(0).values
price_test_return = pd.DataFrame(price_array_test).pct_change(1).fillna(0).values

In [8]:
# print(price_array_train.shape)
# print(price_array_fix.shape)
# print(price_array_validation.shape)
# print(price_array_historical.shape)
# price_historical_return

## Initiate Agent & Environment 

In [9]:
# 金融交易環境for資產配置
class StockPortfolioEnv:  
    
    # 初始化參數由env_args傳入
    def __init__(self, 
                 price_array, #各公司的股價(調整後收盤價) = #all_env_price, #隨機取連續200天的價格
#                  tech_array, #技術指標陣列 = #all_env_tech, #隨機取連續200天的7種技術指標
#                  aug_state_array, #其他的state(embedding / 時間資訊 / 資產資訊) = #all_env_turbulence, #隨機取連續200天的vix                 
                 env_name, #環境名稱
                 lookback, #window size(每次交易看過去幾天) 
                 is_train
                ):
        
        initial_capital = 100000
        self.number = 0
        self.is_train = is_train
        self.lookback = lookback  # 預設看過去10天
        self.initial_total_asset = initial_capital # 初始總資產(股票+現金)
        self.initial_cash = initial_capital #初始現金
        
        self.trans_cost = 0.01 #交易手續費(買賣都一樣)
        self.buy_cost_pct = self.trans_cost # buy手續費
        self.sell_cost_pct = self.trans_cost # sell手續費
        self.gamma = 1 # reward遞減係數
        
        # 載入放入state的陣列
        self.time = lookback-1 #現在的時間點
        self.cash = self.initial_cash
        self.price_array = price_array # (100,82,10)
#         print(price_array.shape)
#         print(price_array)
        self.current_price_array = self.price_array[self.number] # (82,10)
#         print(price_array[self.number])
        self.current_price = self.current_price_array[self.time] # (1 ,10)
#         print(self.current_price_array[self.time])
#         print(self.price_array.shape[2])
        self.stock_num = self.price_array.shape[2] #有幾隻股票要交易
        
        self.max_step =  self.price_array.shape[1] - lookback -1 #每個episode最多走幾步(期初走到最後一天)
        self.env_num = 1 #有幾個環境(預設為1)
        
        
        
        # 價格轉成daily return 
        self.price_return = pd.DataFrame(self.current_price_array).pct_change(1).fillna(0).values # (82,10)
        
#         self.tech_array = tech_array 
#         self.aug_state_array = turbulence_array  
        #self.price_array =  all_env_price
        #self.tech_array = all_env_tech 
        #self.aug_state_array = all_env_turbulence   
        
        
#         self.current_tech = self.tech_array[self.number][self.time] if self.tech_array.shape[0]>0 else self.tech_array
#         self.current_aug_state = self.aug_state_array[self.time] if self.aug_state_array.shape[0]>0 else self.aug_state_array
        self.portfolio = np.zeros(self.stock_num, dtype=np.float32) #現在手上的股票部位
        

#         print('self.price_return',self.price_return.shape)

        
        # 紀錄訓練的結果
        self.history = {'cumu_return':[],'action':[]}

        # 紀錄現有資產及報酬率
#         print('self.current_price:',self.current_price)
        self.total_asset = self.cash + (self.portfolio * self.current_price).sum()
        self.episode_return = 0.0  
        self.gamma_return = 0.0
        

        '''env information'''
        self.env_name = env_name
        self.state_dim = self.price_return.shape[1]*lookback #1 + (self.price_array.shape[1] + self.tech_array.shape[0])*lookback
        self.action_dim = self.price_return.shape[1]+1 #加一維cash
        self.mid_dim = 16 #過完CNN每家公司的embedding長度  
        self.aug_state_dim = 1
        #self.current_working_dir = current_working_dir
        

        self.if_discrete = False
        self.target_return = 10
        self.total_cumulative_return = []
        
    # softmax
    def action_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        
        return softmax_output

    def reset(self) -> np.ndarray:
        self.time = self.lookback-1
        self.current_price = self.current_price_array[self.time]
        
#         self.current_tech = self.tech_array[self.number][self.time] if self.tech_array.shape[0]>0 else self.tech_array
#         self.current_aug_state = self.aug_state_array[self.number][self.time] if self.aug_state_array.shape[0]>0 else self.aug_state_array
        self.cash = self.initial_cash  # reset()
        self.portfolio = np.zeros(self.stock_num, dtype=np.float32) # 持有股數
        self.total_asset = self.cash + np.sum(self.portfolio *  self.current_price) # 總資產價值
        self.history = {'cumu_return':[],'action':[],'total_asset':[],'episode return':[]}
        self.episode_return = 0.0
        self.gamma_return = 0.0
        state  = self.get_state()
        
        return state

    
    def step(self, actions) -> (np.ndarray, float, bool, None):
        self.time += 1
        #print('time:',self.time)
        """transaction"""
        # 隔一天的weight，包含股票及現金的持有比例
        action = self.action_normalization(actions)
        action = action.reshape(-1)
        
        # 新的持有股數：現在的weight*前一天總資產價值/股價(無條件捨去)
        new_portfolio = np.floor(action[:-1]*self.total_asset/self.price_array[self.number][self.time-1])
#         print(new_portfolio)
        # 前一天配置股票剩餘資金納入手上現金部位
        self.cash = self.total_asset - sum(new_portfolio*self.price_array[self.number][self.time-1])
        
        # 計算手續費
        portfolio_change = np.sum((new_portfolio-self.portfolio)*self.price_array[self.number][self.time-1])
        trans_cost = portfolio_change*self.trans_cost
        self.cash-=trans_cost
        
        # 計算新的資產價值
        new_total_asset = np.sum(new_portfolio*self.price_array[self.number][self.time])+self.cash
        #print('cash',self.cash)
        
        # 計算報酬率
        portfolio_return = new_total_asset/self.total_asset
        self.cumu_return = new_total_asset/self.initial_total_asset
        reward = (portfolio_return-1)*100
        
        #更新資產狀態
        self.total_asset = new_total_asset
        self.portfolio = new_portfolio
        
        self.gamma_return = self.gamma_return * self.gamma + reward 
        self.cumu_return = self.total_asset / self.initial_cash
        
#         print('self.cumu_return',self.cumu_return)
        self.history['cumu_return'].append(self.cumu_return)
        self.history['action'].append(action)
        self.history['total_asset'].append(self.total_asset)
                
        """update time"""
        done = self.time == self.max_step+self.lookback
        state = self.get_state()
        if done:
#             print('self.max_step',self.max_step)
            reward = self.gamma_return
#             print('reward',reward)
            self.episode_return = self.total_asset / self.initial_total_asset
            
#             print('.episode_return', self.episode_return)
            self.number += 1
            self.number = self.number % self.price_array.shape[0] 
#             print('self.number:', self.number)
            
            ''' 
            # 畫圖
            plt.plot(self.history['cumu_return'])
            plt.savefig('./train_history/cumulative_return/cumulative_plot'+datetime.now().strftime('%Y-%m-%d %H:%M:%S')+'.png')
            plt.close()
            '''
            
            # 每個episode交易紀錄存檔
            if self.is_train:
                pd.DataFrame(self.history['cumu_return']).to_csv('./train_history(TS0)/cumulative_return/cumu_return'\
                                                                 +datetime.now().strftime("%Y%m%d-%H%M%S")+'.csv')
                pd.DataFrame(self.history['action']).to_csv('./train_history(TS0)/action/action'\
                                                                 +datetime.now().strftime("%Y%m%d-%H%M%S")+'.csv')
            else:
                pd.DataFrame(self.history['cumu_return']).to_csv('./test_history(TS0)/cumulative_return/cumu_return'\
                                                                 +datetime.now().strftime("%Y%m%d-%H%M%S")+'.csv')
                avg_pnl = np.load('./test_history(TS0)/avg_pnl.npy') 
                avg_pnl = np.append(avg_pnl,self.history['cumu_return'][-1])
                np.save('./test_history(TS0)/avg_pnl.npy',avg_pnl)
        return state, reward, done, None

    def get_state(self):
        # 從現在時間往前取n天(lookback)
        state = self.price_return[self.time-self.lookback+1:self.time+1].reshape(-1)*100 #flatten
#         if self.aug_state_array.shape[0]!=0: #如果有aug_state的話
#             state = np.concatenate(state,self.aug_state_array[self.time])

#         if self.tech_array.shape[0]!=0: #如果有tech_array的話
#             for i in range(self.lookback):
#                 tech_i = self.tech_array[self.time-i]
#                 normalized_tech_i = tech_i * 2 ** -15
#                 state = np.hstack((state, normalized_tech_i)).astype(np.float32) 
#         print('state shape',state.shape)    

        return state

### Create Folder

In [10]:
# 清空資料夾
import shutil
import os

if not os.path.exists("./" + 'train_history(TS0)'):
    os.makedirs("./" + 'train_history(TS0)')
shutil.rmtree('./train_history(TS0)')
if not os.path.exists("./" + 'train_history(TS0)/cumulative_return'):
    os.makedirs("./" + 'train_history(TS0)/cumulative_return')    
if not os.path.exists("./" + 'train_history(TS0)/action'):
    os.makedirs("./" + 'train_history(TS0)/action')

if not os.path.exists("./" + 'test_history(TS0)'):
    os.makedirs("./" + 'test_history(TS0)')
shutil.rmtree('./test_history(TS0)')
if not os.path.exists("./" + 'test_history(TS0)/cumulative_return'):
    os.makedirs("./" + 'test_history(TS0)/cumulative_return')
if not os.path.exists("./" + 'PPO_TS_0'):
    os.makedirs("./" + 'PPO_TS_0')
# shutil.rmtree('./PPO_TS_0')
np.save('./test_history(TS0)/avg_pnl.npy',np.array([])) # 清空紀錄檔

## Train and Evaluate RL Agent

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pickle
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import numpy as np

# teacher model
class Sampling_module(nn.Module):
    def __init__(self):
        super(Sampling_module, self).__init__()
        self.layer_1 = nn.Linear(120, 256)
        self.layer_2 = nn.Linear(256, 128)
        self.layer_out = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        return x

learning_rate = 1e-3

sampling_model = Sampling_module()
sampling_optimizer = optim.Adam(sampling_model.parameters(), lr=learning_rate) 

In [12]:
# for j in range(validation_day-rolling_days):
#     all_env_historical = [] #過去60個值
#     for a in range(1281):
#         temp_60 = []
#         for b in days:
#             for c in range(len(ticker_list)):
#                 temp_60.append(price_train_return[j+days[5]+a-b:j+days[5]+a,c].sum())
#         all_env_historical.append(temp_60)
#     all_env_fix = [] #現在60個值
#     for d in days:
#         for e in range(len(ticker_list)):
#             all_env_fix.append(price_fix_return[j+days[5]-d:j+days[5],e].sum()) 
#     value120 = []
#     for k in range(len(all_env_historical)):
#         value120.append(np.concatenate([all_env_historical[k],all_env_fix]))

In [13]:
# current_state = torch.FloatTensor(np.array(value120))  # 計算當前與過去的120個值
# sam_pred = sampling_model(current_state)
# np.set_printoptions(threshold=np.inf)

# act_Softmax = nn.Sigmoid()
# sam_Softmax = act_Softmax(sam_pred)
# print(sam_Softmax.shape)
# # print(sam_Softmax.sum())
# # torch.random.manual_seed(i)
        
# #         print('sam_Softmax:',sam_Softmax[:100])
# pos_idx = np.argsort(sam_Softmax.detach().numpy()[:,0])[-100:]
#         # 計算要給的學生learn的100個環境的idx(pos_idx)，以及沒被取到的1181個idx(neg_index)
# # pos_idx = torch.multinomial(sam_Softmax,100)
# pos_idx

In [14]:
# sampling_model = torch.load('./act/teacher0.pth')
# a = sampling_model.state_dict()
# sampling_model = torch.load('./act/teacher1.pth')
# b = sampling_model.state_dict()

In [15]:
# a['layer_1.weight']-b['layer_1.weight']

In [16]:
# torch.save(sampling_model,'teacher.pth')state_dict

In [17]:
# from torch.distributions import Categorical
# for j in range(validation_day-rolling_days):
#     all_env_historical = [] #過去60個值
#     for a in range(1281):
#         temp_60 = []
#         for b in days:
#             for c in range(len(ticker_list)):
#                 temp_60.append(price_train_return[j+days[5]+a-b:j+days[5]+a,c].sum())
#         all_env_historical.append(temp_60)
#     all_env_fix = [] #現在60個值
#     for d in days:
#         for e in range(len(ticker_list)):
#             all_env_fix.append(price_fix_return[j+days[5]-d:j+days[5],e].sum()) 
#     value120 = []
#     for k in range(len(all_env_historical)):
#         value120.append(np.concatenate([all_env_historical[k],all_env_fix]))
#     action_item = []
#     for f in range(1281):
#         current_state = torch.Tensor(np.array(value120[f]))  # 計算當前與過去的120個值
#         current_state = current_state.to(torch.float32)
#         sam_pred = sampling_model(current_state)
#         act_Softmax = nn.Softmax(dim=0)
#         sam_Softmax = act_Softmax(sam_pred)
#         sam_Softmax = sam_Softmax.view(1, -1)
#         m = Categorical(sam_Softmax)
#         action = m.sample()
#         action_item.append(action.item())
#     np.set_printoptions(threshold=np.inf)
#     print(np.array(action_item))

In [18]:
# np.set_printoptions(threshold=np.inf)

In [19]:
%matplotlib inline
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')
import matplotlib.pyplot as plt
import pylab
#!pip install torch
'''import你要跑的演算法'''
import os
import shutil
from elegantrl.agents.AgentPPO import AgentPPO 
from elegantrl.train.config import Arguments
from elegantrl.train.run import train_and_evaluate,test_agent,train_and_evaluate_mp
import multiprocessing as mp
from sklearn.preprocessing import MinMaxScaler
from torch.distributions import Categorical
from multiprocessing import Process, Pool

env_func = StockPortfolioEnv
env_args = {
    'env_name':'portfolio_allocation',
    'if_discrete' : False,
    'target_return' : 1000,
    'lookback':10,
    'env_num':1,
    'max_step':1000,
    'state_dim':100,
    'action_dim':11,
}

validation_env_args = env_args.copy()
test_env_args = env_args.copy()

epochs = 1000
rolling_days = days[4]
historical_day = price_array_train.shape[0]-days[5] #1281
# TODO：train teacher & student
R = []
r = 0
IDX = np.zeros(1281)
for i in range(100):#teacher_training_epoch
    for j in range(1): #validation_day-rolling_days
        if i%10==0:
            buffer = torch.empty(10)
        prob = torch.empty(1281)
        # copy pretrain好的actor讓student讀
        src=r'./actor.pth'
        des=r'./PPO_TS_0/actor.pth'
        shutil.copy(src, des)
        
        
        # 製作老師的state
        all_env_historical = [] #過去60個值
        for a in range(historical_day):
            temp_60 = []
            for b in days:
                for c in range(len(ticker_list)):
                    temp_60.append(np.prod(price_train_return[j+days[5]+a-b:j+days[5]+a,c]+1)-1)
            all_env_historical.append(temp_60)
        all_env_fix = [] #現在60個值
        for d in days:
            for e in range(len(ticker_list)):
                all_env_fix.append(np.prod(price_fix_return[j+days[5]-d:j+days[5],e]+1)-1) 
        value120 = []
        for k in range(len(all_env_historical)):
            value120.append(np.concatenate([all_env_historical[k],all_env_fix]))

        
        # 老師打分數，取前100個高的
        current_state = torch.FloatTensor(np.array(value120))  # 計算當前與過去的120個值
        sam_pred = sampling_model(current_state)
#         act_Softmax = nn.Softmax()
#         sam_Softmax = act_Softmax(sam_pred)
        torch.random.manual_seed(i)
        
        
        
        act_Softmax = nn.Softmax(dim=1)
        sam_Softmax = act_Softmax(sam_pred)
#         pos_idx = np.argsort(sam_Softmax.detach().numpy()[:,0])[-100:]
#         print('sam_Softmax:',sam_Softmax[:100])
        
        # 計算要給的學生learn的100個環境的idx(pos_idx)，以及沒被取到的1181個idx(neg_index)
        action_item = np.ones(historical_day)
        pos_idx = sam_Softmax[:,0].multinomial(num_samples=100, replacement=False)
        IDX[pos_idx]+=1
        for g in pos_idx:
            action_item[g] = 0
        for h in range(historical_day):
            prob[h] = torch.log(sam_Softmax[h][int(action_item[h])])
#         neg_index = list(set(range(1281))-set(pos_idx))
#         action = sam_Softmax[:,1]
#         action = np.arange(1281)
#         for i in range(1281):
#             if i in pos_idx:
#                 action[i] = sam_Softmax[i,0]
#             else:
#                 action[i] = sam_Softmax[i,1]
#         pos_log_prob = torch.log(torch.Tensor(action))
#         pos_log_prob = torch.log(sam_Softmax[pos_idx,0])
#         neg_log_prob = torch.log(sam_Softmax[neg_index,1])
        
        
#         print('pos_idx',pos_idx)
        all_env_train = []
        for f in range(len(pos_idx)): #100
            all_env_train.append(price_array_historical[pos_idx[f]+days[5]:pos_idx[f]+days[4]+days[5], :])
#         print("train")
        train_env_args = env_args.copy()
        train_env_args.update({
            'price_array' : np.array(all_env_train),
            'is_train' : True
        })
        train_args = Arguments(AgentPPO, env_func=env_func, env_args=train_env_args,cwd = "PPO_TS_0")
        train_and_evaluate(train_args)

        fileDir = r"PPO_TS_0"
        fileSxt = r"actor"
        fileExt = r".pth"
        direction = [_ for _ in os.listdir(fileDir) if _.startswith(fileSxt) and _.endswith(fileExt) ]
        direction.sort()
#         print(direction[-1])
#         os.rename(f'./PPO_TS_0/{direction[-1]}',f'./PPO_TS_0/actor.pth')
        shutil.move(f'./PPO_TS_0/{direction[-1]}', f'./PPO_TS_0/actor.pth')

        direction = [_ for _ in os.listdir(fileDir) if _== 'act_optim.pth' or _== 'act_target.pth']
        if direction:
            for file in direction:
                os.remove(f'./PPO_TS_0/{file}')
        

        # 丟到驗證環境得到給老師的reward
        validation_env = price_array_validation[j:j+days[4]]
        validation_env_args.update({
            'price_array' : np.expand_dims(np.array(validation_env), axis=0),
#             'tech_array':tech_array_test,
#             'turbulence_array':turbulence_array_test,
#             'aug_state_array' : aug_state_array_test,
            'is_train' : False,
        })
#         print("validation")
        validation_args = Arguments(AgentPPO, env_func=env_func, env_args=validation_env_args,cwd = "PPO_TS_0")
        test_agent(validation_args)
        
        reward = np.load('./test_history(TS0)/avg_pnl.npy')
        R.append(reward)
        np.save('./test_history(TS0)/avg_pnl.npy',np.array([])) # 清空紀錄檔
        print(i)
        
        plt.title('Reward')
        plt.plot(R)
        plt.autoscale()
        pylab.show()
        plt.savefig('./Reward.png',bbox_inches='tight')
        
        plt.cla()
        plt.title('Prob')
        plt.plot(sam_Softmax[:,0].detach().numpy())
        plt.autoscale()
        pylab.show()
        plt.savefig('./Prob.png',bbox_inches='tight')
        
        plt.cla()
        plt.title('Idx')
        plt.plot(IDX)
        plt.autoscale()
        plt.savefig('./Idx.png',bbox_inches='tight')
        
        plt.cla()
        Prob = sam_Softmax[:,0].detach().numpy()
        fig = plt.figure()
        ax = fig.add_subplot(111)
        base = np.array([IDX.max()]*1281)
        new = base+Prob
        ax.plot(Prob,'r')
        ax2 = ax.twinx()
        ax2.plot(IDX)
        ax.set_ylim(Prob.min(),Prob.max())
        ax2.set_ylim(IDX.min()-1, IDX.max()+1)
        plt.savefig('./Idx&Prob.png',bbox_inches='tight')
        
#         print('grad:',sampling_model.layer_1.weight.grad)
        torch.set_grad_enabled(True)
        
        buffer[i%10] = (-1*prob*torch.FloatTensor((R[r]-np.mean(R)))).mean()
        l=(-1*prob*torch.FloatTensor((R[r]-np.mean(R)))).mean()
        print('prob',prob.grad_fn)
        print('loss.grad_fn',l.grad_fn)
        print('l ',l)
#         pos_loss = (-1*pos_log_prob*torch.FloatTensor((R[r]-1.035))).mean()
#         neg_loss = (-1*neg_log_prob*torch.FloatTensor((R[r]-np.mean(R)))).mean()
        r += 1
        if (i-9)%10==0:
            print(buffer.sum())
            buffer.sum().backward()
#             print('grad:',sampling_model.layer_1.weight.grad)
            print('buffer_grad',buffer.grad)
            sampling_optimizer.step()
            sampling_optimizer.zero_grad()
            print("update")
#         pos_loss.backward()
#         torch.autograd.set_detect_anomaly(True)
#         if i%10==0:
#             sampling_optimizer.zero_grad()
#             sampling_optimizer.step()
        
#         print('pos_loss:',pos_loss.item())
#         print('neg_loss:',neg_loss.item())
        
#         neg_loss.backward()
#         sampling_optimizer.step()
#         sampling_optimizer.zero_grad()
        
#         torch.save(sampling_model,f'./act/teacher{j}.pth')
    
    
print('=====testing=====')

# test_agent(test_args)

In [ ]:
buffer_grad = Variable(torch.Tensor(buffer),requires_grad=True)
reward = np.load('./test_history(TS0)/avg_pnl.npy')
print("reward:",reward.shape)
R.append(reward)
print("R:",R)
np.save('./test_history(TS0)/avg_pnl.npy',np.array([])) # 清空紀錄檔
print("R_mean:",np.mean(R))
neg_pred = -1 * (R[r]-np.mean(R)) * buffer_grad #更新的reward #R有正有負
r += 1
neg_reinforce_loss = torch.sum(neg_pred).mean()
sampling_optimizer.zero_grad()
torch.autograd.set_detect_anomaly(True)
neg_reinforce_loss = Variable(neg_reinforce_loss,requires_grad=True)
neg_reinforce_loss.backward(retain_graph=True)
sampling_optimizer.step()

In [20]:
from elegantrl.agents.AgentPPO import AgentPPO 
from elegantrl.train.config import Arguments
from elegantrl.agents.AgentBase import AgentBase
from elegantrl.train.run import train_and_evaluate,test_agent
import importlib
from elegantrl.agents import AgentBase as agentbase
import elegantrl
importlib.reload(elegantrl.agents)
price_array_test = pd.read_csv('./data/test.csv').to_numpy()
price_array_test = np.expand_dims(price_array_test,axis=0)
test_env_args.update({
    'price_array' : price_array_test,
#     'tech_array':tech_array_test,
#     'turbulence_array':turbulence_array_test,
#     'aug_state_array' : aug_state_array_test,
    'is_train' : False
})
# print(all_env100_test.shape)
test_args = Arguments(AgentPPO, env_func=env_func, env_args=test_env_args,cwd = "PPO_TS_0")
test_agent(test_args)

cwd: PPO_TS_0
_path: PPO_TS_0/actor.pth


In [ ]:
model = torch.load('./actor.pth')
model

In [ ]:
import os
os.remove('UBAH_test.csv')
ubah = pd.DataFrame(price_array_test[0]).pct_change().fillna(0)+1
ubah.cumprod().mean(axis=1).to_csv('UBAH_test.csv')

### plot training result

In [ ]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
name = []
cumu_return = []
count = 0
for root,_,files in os.walk('./train_history/cumulative_return'):
    for i in files:
        if i.endswith('csv') and 'checkpoint' not in i:
            name.append(i)   
            
name.sort()
for i in name:
    count+=1
    tmp = pd.read_csv(os.path.join('./train_history/cumulative_return',i))
    cumu_return.append(tmp['0'].values[-1])
            
print('total episode:',count)
ma = pd.DataFrame(cumu_return).rolling(50).mean()
plt.title('episode cumulative return in training period')
plt.ylabel('cumu return')
plt.xlabel('number of episode')
plt.plot(cumu_return)
plt.plot(ma)
plt.show()

In [ ]:
import os
root = './train_history/action/'
action = os.listdir(root)[-1]
df = pd.read_csv(os.path.join(root,action))

In [ ]:
'''
1. df是每個時間點的action(portfolio weight)
2. price_array是每家公司的股價
3. ticker list是每間公司的代號(只是用來顯示在圖片的title而已)
'''

%matplotlib inline
for num,name in enumerate(ticker_list):
    
    fig = plt.figure()
    fig.set_figheight(15)
    fig.set_figwidth(20)
    ax = fig.add_subplot(111)
    Y = price_array_test[11:,num]
    X = range(len(Y))
    buy_sig = df.iloc[:,num+1].diff().fillna(0)>0.1  # 大於10%才有箭頭 
    sell_sig = df.iloc[:,num+1].diff().fillna(0)<-0.1
    ax.plot(X,Y)
    for i in X:
        if buy_sig[i]:
            #print('buy')
            ax.annotate("", xy=(i, Y[i]-0.1),xytext=(i, Y[i]-0.3), arrowprops=dict(facecolor="g", alpha=0.5, headlength=4, width=0.1)) 
        elif sell_sig[i]:
            #print('sell')
            ax.annotate("", xy=(i, Y[i]+0.1),xytext=(i, Y[i]+0.3),arrowprops=dict(facecolor="r", alpha=0.5, headlength=4, width=0.1)) 
    plt.title(name)
    plt.show()


In [ ]:
import random

a=random.sample(range(price_array_train.shape[0]), 50)
a

### BackTest

